In [1]:
import os
import sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math

In [21]:
#number of atoms
natoms = 9
nfreqs = 252
allfreqs = 252#int((nfreqs**2+nfreqs*3)/2)

In [22]:
int((nfreqs**2+nfreqs*3)/2)

32130

In [23]:
#Read structure in A and atoms from Gaussian input file
with open('freq_anharm.gjf', 'r') as f:
    lines = f.readlines()
struc_A_IM = lines[-natoms:]
atoms = np.zeros(natoms,str)
struc_A = np.zeros((natoms,3))
for nr,i in enumerate(struc_A_IM):
    atoms[nr] = i.split()[0]
    struc_A[nr] = i.split()[1:]

In [24]:
#Read structure in A from Gaussian input file
with open('freq_anharm.fch', 'r') as f:
    for line in f:
        if 'Current cartesian coordinates' in line:
            struc_au_IM = f.readlines()[:math.ceil(natoms*3 / 5)]
            break

struc_au = []
for i in struc_au_IM:
    struc_au = struc_au + i.split()
struc_au = np.array(struc_au,dtype=float).reshape(natoms,3)

In [25]:
#Read vibrational energies
with open('freq_anharm.fch', 'r') as f:
    for line in f:
        if 'Anharmonic Vib-E2' in line:
            data_IM = f.readlines()[:math.ceil( allfreqs*4 / 5)]
            break

data = []
energies = []
IR = []
for i in data_IM:
    data = data + i.split()
energies = np.array(data[:nfreqs],dtype=float)
IR = np.array(data[allfreqs*3:(allfreqs*3+nfreqs)],dtype=float)

In [26]:
#Read anharmonic modes
with open('freq_anharm.fch', 'r') as f:
    for line in f:
        if 'Anharmonic Vib-Modes' in line:
            data_IM = f.readlines()[:math.ceil( nfreqs*natoms*3 / 5)]
            break
data = []
for i in data_IM:
    data = data + i.split()
data = data[:nfreqs*natoms*3]
modes = np.array(data,dtype=float).reshape(nfreqs,natoms,3)

In [27]:
sortlist = np.argsort(energies)
#sortlist = np.arange(nfreqs)

In [37]:
with open ('freq_anharm.molden', 'w') as f:
    f.write('[Molden Format]\n')
    f.write('[GEOMETRIES] XYZ\n')
    f.write('%5i' %  natoms+'\n')
    f.write('\n')
    for i in range(natoms):
        f.write(' ' + atoms[i] + '    ' + "%9.6f" % struc_A[i,0] + '    ' + "%9.6f" % struc_A[i,1] +'    ' +  "%9.6f" % struc_A[i,2] + '\n' )
    f.write('[FREQ]\n')
    for i in sortlist:#range(nfreqs):
        f.write('%10.4f' % energies[i] + '\n')
    f.write('[INT]\n')
    for i in sortlist:#range(nfreqs):
        f.write('%10.4f' % IR[i] + '     0.0000\n')
    f.write('[FR-COORD]\n')
    for i in range(natoms):
        f.write(' ' + atoms[i] + '    ' + "%9.6f" % struc_au[i,0] + '    ' + "%9.6f" % struc_au[i,1] +'    ' +  "%9.6f" % struc_au[i,2] + '\n' )
    f.write('[FR-NORM-COORD]\n')
    for nr,i in enumerate(sortlist):#range(nfreqs):
        f.write('vibration ' + '%5i' % (nr+1) + '\n')
        for j in range(natoms):
            f.write("%12.6f" % modes[i,j,0] + ' ' + "%12.6f" % modes[i,j,1] + ' ' +  "%12.6f" % modes[i,j,2] + '\n' )

In [29]:
np.shape(modes)

(252, 9, 3)

In [30]:
mass_c = 12.000000 
mass_o = 15.994915 
mass_h = 1.007825 
molecule = [mass_c] * 4 + [mass_o] + [mass_h] * 4

In [31]:
molecule

[12.0, 12.0, 12.0, 12.0, 15.994915, 1.007825, 1.007825, 1.007825, 1.007825]

In [32]:
norm = np.zeros(nfreqs)
for imode in range(nfreqs):
    norm[imode] = 0.0
    #calculate norm
    for j, atom in enumerate(molecule):
        for xyz in range(3):
            norm[imode] +=  modes[imode,j,xyz]**2*atom
    norm[imode] = math.sqrt(norm[imode])
    #print(norm[imode])
print(norm)

[1.18269715 2.75831722 1.68502209 2.16470956 1.30126338 1.64833228
 1.21567896 1.2438388  1.10040643 1.14737284 1.19677146 1.06430531
 1.22686318 1.0421685  1.2005313  1.22819027 1.07515692 1.16969837
 1.10512271 1.14165135 1.07110058 2.00639987 1.53787829 2.00122616
 1.41557138 1.41585816 2.00992778 1.41563121 1.41882186 1.42003335
 2.02947861 1.41528259 1.42204661 1.49482633 1.4212432  2.00168234
 1.42425664 1.41577844 1.42199422 1.42454643 1.41904065 2.00209942
 1.41405367 1.4173457  1.40651493 1.41338487 1.4178248  1.41181846
 1.99688176 1.41422739 1.41422208 1.41424648 1.41426314 1.41422666
 1.41423515 1.41127374 2.00011117 1.4172417  1.41224732 1.41664019
 1.41495574 1.41692972 1.40981094 1.41380508 1.41790723 1.99333908
 1.41425251 1.41422645 1.41427761 1.41425015 1.41425847 1.41423089
 1.40946807 1.4144117  1.41963514 2.00066379 1.41509358 1.41589984
 1.41184475 1.4137794  1.4182995  1.41301052 1.4155101  1.40898925
 1.41065678 1.40866253 1.96516289 1.40758648 1.41341145 1.4064

In [33]:
for imode in range(nfreqs):
    for j, atom in enumerate(molecule):
        for xyz in range(3):
            modes[imode,j,xyz] /= norm[imode]/math.sqrt(atom)

In [34]:
results = np.transpose(modes.reshape(nfreqs,27))

In [35]:
np.dot(results.T,results)

array([[ 1.00000000e+00, -9.06917756e-03, -7.95223694e-02, ...,
         8.82070222e-08,  4.11298470e-07,  7.94948014e-08],
       [-9.06917756e-03,  1.00000000e+00,  5.23273298e-04, ...,
         4.36357496e-08, -3.27727787e-08, -1.87797477e-08],
       [-7.95223694e-02,  5.23273298e-04,  1.00000000e+00, ...,
        -8.02062505e-07, -9.44036570e-07,  6.07670707e-07],
       ...,
       [ 8.82070222e-08,  4.36357496e-08, -8.02062505e-07, ...,
         1.00000000e+00,  4.58205404e-01,  5.25503739e-01],
       [ 4.11298470e-07, -3.27727787e-08, -9.44036570e-07, ...,
         4.58205404e-01,  1.00000000e+00,  1.10136150e-01],
       [ 7.94948014e-08, -1.87797477e-08,  6.07670707e-07, ...,
         5.25503739e-01,  1.10136150e-01,  1.00000000e+00]])

In [36]:
import pandas
from pandas import DataFrame
print(DataFrame(np.dot(results.T,results)))

              0             1             2             3             4    \
0    1.000000e+00 -9.069178e-03 -7.952237e-02  5.365216e-04 -2.096963e-02   
1   -9.069178e-03  1.000000e+00  5.232733e-04  7.175276e-01  5.130301e-01   
2   -7.952237e-02  5.232733e-04  1.000000e+00  2.744779e-04  7.871836e-04   
3    5.365216e-04  7.175276e-01  2.744779e-04  1.000000e+00  5.674840e-01   
4   -2.096963e-02  5.130301e-01  7.871836e-04  5.674840e-01  1.000000e+00   
..            ...           ...           ...           ...           ...   
247 -6.812423e-06 -3.489932e-08  4.329269e-06  1.682741e-06 -2.048199e-06   
248 -4.829832e-07 -3.824916e-08 -1.146609e-06 -1.461387e-07  1.730004e-07   
249  8.820702e-08  4.363575e-08 -8.020625e-07 -3.614378e-07  8.502918e-08   
250  4.112985e-07 -3.277278e-08 -9.440366e-07  2.774695e-07 -2.023434e-07   
251  7.949480e-08 -1.877975e-08  6.076707e-07  5.212001e-07 -5.752012e-07   

              5         6         7         8         9    ...       242  \